## Backtest Orchestrator Workflow

This notebook automates a rolling-window backtest by iteratively executing a worker notebook (`py11_worker_sharpe_strategy.ipynb`) for each time slice.

**Workflow:**

1.  **Setup:** Loads project configurations and strategy parameters.
2.  **Load Data:** Reads the master adjusted close prices file.
3.  **Prepare Data Chunks:** Creates rolling window data chunks for the entire dataset.
4.  **Split Data:** Divides the chunks into an **In-Sample** set (for the main walk-forward backtest) and a **Holdout** set (for final, unseen data verification).
5.  **Execute In-Sample Backtests:** Loops through the **In-Sample** chunks, running the worker notebook for each one.
6.  **Aggregate In-Sample Results:** Collects and combines the results from the in-sample backtest into a single portfolio returns file.
7.  **Cleanup:** Deletes temporary data files.
8.  **Next Steps:** Outlines how to use the **Holdout** data for a final, true out-of-sample validation.

## Step 1: Setup and Configuration

This cell contains all imports and configuration variables. It defines the project structure and parameters for the rolling backtest.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import papermill as pm
import shutil
from IPython.display import display, Markdown

# --- Project Path Configuration ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
TEMP_DIR = NOTEBOOK_DIR / 'temp_backtest_data'
OUTPUT_DIR = NOTEBOOK_DIR / 'backtest_results'

# --- Add src to Python path ---
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- Import Custom Modules ---
import utils

# --- Papermill Configuration ---
WORKER_NOTEBOOK_NAME = "_pm_worker_sharpe_strategy.ipynb"
WORKER_NOTEBOOK_PATH = NOTEBOOK_DIR / WORKER_NOTEBOOK_NAME
IN_SAMPLE_RESULTS_FILENAME = "in_sample_portfolio_returns.parquet"
# [NEW] A separate filename for the eventual holdout results
HOLDOUT_RESULTS_FILENAME = "holdout_portfolio_returns.parquet"

# --- Strategy & Backtest Parameters ---
SLIDING_WINDOW_WIDTH = 300
SLIDING_WINDOW_STEP = 30
TRAIN_TEST_SPLIT_POINT = 270
BENCHMARK_TICKER = 'VGT'
FINVIZ_DATA_FILENAME = '2025-08-01_df_finviz_merged_stocks_etfs.parquet'
# [NEW] Ratio of chunks to reserve for the final holdout test. 0.25 means the last 25% of chunks are held out.
HOLDOUT_SPLIT_RATIO = 0.25
BACKTEST_CHUNK_LIMIT = 2

# --- Verification ---
print("--- Path Configuration ---")
print(f"ROOT_DIR:                {ROOT_DIR}")
print(f"NOTEBOOK_DIR:            {NOTEBOOK_DIR}")
print(f"DATA_DIR:                {DATA_DIR}")
print(f"SRC_DIR:                 {SRC_DIR}")
print(f"TEMP_DIR:                {TEMP_DIR}")
print(f"OUTPUT_DIR:              {OUTPUT_DIR}")
print(f"WORKER_NOTEBOOK_PATH:    {WORKER_NOTEBOOK_PATH}")
assert all([ROOT_DIR.exists(), DATA_DIR.exists(), SRC_DIR.exists(), NOTEBOOK_DIR.exists()]), "A key directory was not found!"
assert (DATA_DIR / FINVIZ_DATA_FILENAME).exists(), f"Finviz data file not found: {FINVIZ_DATA_FILENAME}"
assert WORKER_NOTEBOOK_PATH.exists(), f"Worker notebook not found at: {WORKER_NOTEBOOK_PATH}"

print("\n--- Strategy Parameters ---")
print(f"Window Width:  {SLIDING_WINDOW_WIDTH}, Step Size: {SLIDING_WINDOW_STEP}")
print(f"In-Sample/Holdout Split Ratio: {HOLDOUT_SPLIT_RATIO}")
print(f"Train/Test Split: {TRAIN_TEST_SPLIT_POINT} rows for training")
print(f"Benchmark:     {BENCHMARK_TICKER}")
if BACKTEST_CHUNK_LIMIT is None:
    print(f"Chunk Limit:   None (run all in-sample chunks)")
else:
    print(f"Chunk Limit:   {BACKTEST_CHUNK_LIMIT} (for testing)")

--- Path Configuration ---
ROOT_DIR:                c:\Users\ping\Files_win10\python\py311\stocks
NOTEBOOK_DIR:            c:\Users\ping\Files_win10\python\py311\stocks\colab
DATA_DIR:                c:\Users\ping\Files_win10\python\py311\stocks\data
SRC_DIR:                 c:\Users\ping\Files_win10\python\py311\stocks\src
TEMP_DIR:                c:\Users\ping\Files_win10\python\py311\stocks\colab\temp_backtest_data
OUTPUT_DIR:              c:\Users\ping\Files_win10\python\py311\stocks\colab\backtest_results
WORKER_NOTEBOOK_PATH:    c:\Users\ping\Files_win10\python\py311\stocks\colab\_pm_worker_sharpe_strategy.ipynb

--- Strategy Parameters ---
Window Width:  300, Step Size: 30
In-Sample/Holdout Split Ratio: 0.25
Train/Test Split: 270 rows for training
Benchmark:     VGT
Chunk Limit:   2 (for testing)


### Step 2: Load and Prepare Data

Load the adjusted close prices and convert them to percentage returns, which form the basis of our analysis.

In [2]:
# Load historical price data
adj_close_path = DATA_DIR / 'df_adj_close.parquet'
df_adj_close = pd.read_parquet(adj_close_path)

# Calculate daily returns
returns = df_adj_close.pct_change().dropna()

# Add a risk-free 'CASH' asset with zero return
returns['CASH'] = 0.0

print(f"Loaded returns data. Shape: {returns.shape}")
print(f"Date range: {returns.index.min().strftime('%Y-%m-%d')} to {returns.index.max().strftime('%Y-%m-%d')}")
display(returns.head(3))

Loaded returns data. Shape: (2662, 1220)
Date range: 2015-01-05 to 2025-08-05


Ticker,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABT,ACGL,ACM,...,XYL,YPF,YUM,ZBH,ZBRA,ZG,ZION,ZTS,ZWS,CASH
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.018739,-0.057934,-0.000556,-0.033745,-0.028174,-0.018821,-0.018488,0.000224,-0.005987,-0.043822,...,-0.062237,-0.051703,-0.020317,0.037311,-0.014077,-0.058835,-0.037471,-0.006004,-0.039730,0.0
2015-01-06,-0.015579,0.007352,-0.015590,-0.016051,0.000097,-0.004950,0.037673,-0.011356,0.002236,-0.009992,...,-0.005883,-0.003230,-0.012275,-0.008484,-0.007205,-0.006560,-0.038192,-0.009755,-0.016766,0.0
2015-01-07,0.013273,0.025880,-0.000565,0.009112,0.014022,0.040417,0.016500,0.008108,0.005837,0.020536,...,0.007887,-0.013373,0.033138,0.024875,0.025465,0.049309,0.009547,0.020643,0.001513,0.0


## Step 3: Generate Rolling Window Chunks

Using our utility function, slice the returns data into overlapping chunks that will be used for each iteration of the backtest.

In [3]:
# (This cell remains exactly the same as before)
rolling_chunks = utils.create_rolling_window_chunks(
    returns_df=returns,
    window_width=SLIDING_WINDOW_WIDTH,
    step_size=SLIDING_WINDOW_STEP
)

print(f"Successfully generated {len(rolling_chunks)} total rolling window chunks.")
if rolling_chunks:
    chunk = rolling_chunks[0]
    print(f"Shape of each chunk: {chunk.shape}")

Successfully generated 79 total rolling window chunks.
Shape of each chunk: (300, 1220)


### Step 4: Split Data into In-Sample and Holdout Sets

Divide the generated chunks into a primary set for the walk-forward backtest (`in_sample_chunks`) and a final validation set (`holdout_chunks`) that will remain untouched during this run.

In [4]:
# [NEW] This is the new cell implementing the requested logic.
if not rolling_chunks:
    raise ValueError("No rolling chunks were generated. Cannot proceed with split.")

# Calculate the index at which to split the chunks
split_index = int(len(rolling_chunks) * (1 - HOLDOUT_SPLIT_RATIO))

# Assign chunks to their respective sets
in_sample_chunks = rolling_chunks[:split_index]
holdout_chunks = rolling_chunks[split_index:]

print("--- Data Split Summary ---")
print(f"Total chunks:      {len(rolling_chunks)}")
print(f"In-Sample chunks:  {len(in_sample_chunks)} (for immediate backtesting)")
print(f"Holdout chunks:    {len(holdout_chunks)} (reserved for final validation)")

if in_sample_chunks:
    print(f"In-Sample Period:  {in_sample_chunks[0].index.min().date()} to {in_sample_chunks[-1].index.max().date()}")
if holdout_chunks:
    print(f"Holdout Period:    {holdout_chunks[0].index.min().date()} to {holdout_chunks[-1].index.max().date()}")

--- Data Split Summary ---
Total chunks:      79
In-Sample chunks:  59 (for immediate backtesting)
Holdout chunks:    20 (reserved for final validation)
In-Sample Period:  2015-01-05 to 2023-02-09
Holdout Period:    2022-01-13 to 2025-07-03


### Step 5: Select Chunks for This Run

This cell applies the `BACKTEST_CHUNK_LIMIT` to select the subset of in-sample chunks that will be processed in this execution. This is useful for running quick tests without processing the entire dataset.


In [5]:
# [NEW] Create the list of chunks we will actually process based on the limit.
if BACKTEST_CHUNK_LIMIT is None:
    chunks_to_process = in_sample_chunks
    print(f"BACKTEST_CHUNK_LIMIT is not set. Processing all {len(in_sample_chunks)} in-sample chunks.")
elif not in_sample_chunks:
    chunks_to_process = []
    print("Warning: No in-sample chunks available to process.")
else:
    chunks_to_process = in_sample_chunks[:BACKTEST_CHUNK_LIMIT]
    print(f"BACKTEST_CHUNK_LIMIT is set to {BACKTEST_CHUNK_LIMIT}. Processing the first {len(chunks_to_process)} of {len(in_sample_chunks)} available in-sample chunks.")

BACKTEST_CHUNK_LIMIT is set to 2. Processing the first 2 of 59 available in-sample chunks.


### Step 6: Execute In-Sample Backtest Loop via Papermill

Iterate through the **in-sample** data chunks, save the train/test splits, and execute the worker notebook. The holdout data is not used here.

In [6]:
# [MODIFIED] This step now operates only on `in_sample_chunks`.
TEMP_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"--- Starting Papermill Execution Loop for {len(in_sample_chunks)} In-Sample chunks ---")

# The loop now iterates over in_sample_chunks
for i, chunk in enumerate(in_sample_chunks[:2]):
    # The rest of the logic inside the loop is the same, but the 'i' now corresponds
    # to the index within the in_sample_chunks list.
    returns_train = chunk.iloc[:TRAIN_TEST_SPLIT_POINT]
    returns_test = chunk.iloc[TRAIN_TEST_SPLIT_POINT:]

    train_file = TEMP_DIR / f"returns_train_chunk_{i}.parquet"
    test_file = TEMP_DIR / f"returns_test_chunk_{i}.parquet"
    result_file = OUTPUT_DIR / f"result_chunk_{i}.parquet"
    output_notebook_path = OUTPUT_DIR / f"output_notebook_chunk_{i}.ipynb"

    returns_train.to_parquet(train_file)
    returns_test.to_parquet(test_file)

    print(f"\nExecuting In-Sample chunk {i+1}/{len(in_sample_chunks)}...")
    # ... (rest of the Papermill execution logic is identical)
    pm.execute_notebook(
       input_path=WORKER_NOTEBOOK_PATH,
       output_path=output_notebook_path,
       parameters={
           "returns_train_path": str(train_file),
           "returns_test_path": str(test_file),
           "finviz_data_path": str(DATA_DIR / FINVIZ_DATA_FILENAME),
           "output_path": str(result_file),
           "benchmark_ticker": BENCHMARK_TICKER,
       },
       kernel_name="python3"
    )

print("\n--- In-Sample Papermill execution complete. ---")


--- Starting Papermill Execution Loop for 59 In-Sample chunks ---

Executing In-Sample chunk 1/59...


Executing:   0%|          | 0/17 [00:00<?, ?cell/s]


Executing In-Sample chunk 2/59...


Executing:   0%|          | 0/17 [00:00<?, ?cell/s]


--- In-Sample Papermill execution complete. ---


### Step 7: Aggregate and Save In-Sample Results

Collect the individual result files from the in-sample run and combine them into a single, continuous time series.


In [7]:
# [MODIFIED] This step now correctly aggregates the results from the in-sample run.
all_results = []
# The loop range must match the number of chunks processed in the previous step
for i in range(len(in_sample_chunks[:2])):
    result_file = OUTPUT_DIR / f"result_chunk_{i}.parquet"
    if result_file.exists():
        chunk_result = pd.read_parquet(result_file)
        all_results.append(chunk_result)
    else:
        print(f"Warning: Result file not found for chunk {i}: {result_file}")

if all_results:
    in_sample_returns = pd.concat(all_results).sort_index()
    in_sample_returns = in_sample_returns[~in_sample_returns.index.duplicated(keep='first')]

    # Save the aggregated in-sample results
    final_output_path = OUTPUT_DIR / IN_SAMPLE_RESULTS_FILENAME
    in_sample_returns.to_parquet(final_output_path)

    print(f"Successfully aggregated {len(all_results)} in-sample result chunks.")
    print(f"Final in-sample portfolio returns saved to: {final_output_path}")
    display(in_sample_returns.head())
    display(in_sample_returns.tail())
else:
    print("No result files found to aggregate.")

Successfully aggregated 2 in-sample result chunks.
Final in-sample portfolio returns saved to: c:\Users\ping\Files_win10\python\py311\stocks\colab\backtest_results\in_sample_portfolio_returns.parquet


,Portfolio,Benchmark
Date,,
2016-02-01,-0.002675,0.002453
2016-02-02,-0.031677,-0.020259
2016-02-03,0.008096,-0.002297
2016-02-04,0.015184,0.002403
2016-02-05,-0.021902,-0.037456


,Portfolio,Benchmark
Date,,
2016-04-20,0.008825,0.002465
2016-04-21,0.002819,-0.000727
2016-04-22,0.008572,-0.014961
2016-04-25,-0.007720,-0.001945
2016-04-26,0.006864,-0.001763


### Step 8: Cleanup Temporary Files
Remove the intermediate temp_backtest_data directory to keep the project folder clean. This cell includes a retry mechanism to handle potential file-locking issues, especially on Windows.

In [8]:
import shutil
import gc
import time

# --- Cleanup Configuration ---

CLEANUP_RETRIES = 5
CLEANUP_DELAY_SECONDS = 3

# --- Execution ---

if TEMP_DIR.exists():
  print(f"Attempting to remove temporary directory: {TEMP_DIR}")
  for attempt in range(CLEANUP_RETRIES):
      # First, run garbage collection to help release any Python-level object locks
      gc.collect()
      
      try:
          shutil.rmtree(TEMP_DIR)
          print(f"✅ Successfully removed temporary directory on attempt {attempt + 1}.")
          break # Exit the loop if successful
      except OSError as e:
          print(f"Attempt {attempt + 1}/{CLEANUP_RETRIES} failed: {e}")
          if attempt < CLEANUP_RETRIES - 1:
              print(f"Waiting {CLEANUP_DELAY_SECONDS} seconds before retrying...")
              time.sleep(CLEANUP_DELAY_SECONDS)
          else:
              print(f"❌ Error: Could not remove temporary directory after {CLEANUP_RETRIES} attempts.")
              print("A file may still be locked by another process. You may need to restart the kernel and delete the directory manually.")
  else:
      print("Temporary directory not found, no cleanup needed.")

Attempting to remove temporary directory: c:\Users\ping\Files_win10\python\py311\stocks\colab\temp_backtest_data
✅ Successfully removed temporary directory on attempt 1.


### Step 9: Verifying on Holdout Data (Next Steps)

The `holdout_chunks` have been preserved and were not used in the backtest above. They can now be used for a true out-of-sample test of the final strategy. This would typically be done in a separate verification notebook (`py12_holdout_verification.ipynb`) to maintain a clean workflow.

A simplified version of that process would look like this:

```python
# --- PSEUDO-CODE for a future holdout test ---

# # 1. Loop through the holdout_chunks
# for i, chunk in enumerate(holdout_chunks):
#     # The chunk index 'i' would need to be offset to avoid overwriting temp files
#     # if run in the same script.
#     chunk_index_offset = len(in_sample_chunks) + i
#
#     # 2. Run papermill exactly as before on each holdout chunk
#     pm.execute_notebook(...)
#
# # 3. Aggregate the holdout results into a separate file
# holdout_results = pd.concat(...)
# holdout_results.to_parquet(OUTPUT_DIR / HOLDOUT_RESULTS_FILENAME)
#
# # 4. Compare the performance metrics (Sharpe, CAGR, Drawdown) of the
# #    in_sample_returns vs. the holdout_returns to check for overfitting.
```